In [45]:
import gcp.bigquery as bq
import pandas as pd
import numpy as np
import scipy


import matplotlib.pyplot as plt

from sklearn import datasets, svm
from sklearn.feature_selection import SelectPercentile, f_classif

Lets split the dataset into fraud and non fraud

In [8]:
%%sql --module frauds
SELECT * FROM [FRAUD_US.KB_FRAUD_OUTPUT] 
where session_start is not null

In [24]:
%%sql --module nonfrauds
SELECT *, rand(5) as rand
FROM [FRAUD_US.KB_FRAUD_OUTPUT] 
where session_start is null
limit 50000

In [26]:
%%bigquery sample --count 5 --query nonfrauds

PAGE_VIEW_TS,PAGE_VIEW_DT,PAGE_VIEW_TM,COOKIE_FIRST_TX,COOKIE_THIRD_TX,USER_ID,IP_HASH_CD,DATA_CLLCTN_SERVER_NR,DATA_CLLCTN_SERVER_CD,SSL_IN,HOST_NR,URI_TX,URL_TX,RAW_URI_TX,RAW_URL_TX,REFRRL_TX,TAG_VERSIN_NR,AD_IMPRSN_TX,AD_CLICK_TX,ERROR_TX,PLTFRM_TX,FUNNEL_NR,FUNNEL_STEP_NO,FUNNEL_STEP_CD,FUNNEL_STEP_NR,FUNNEL_PRODCT_NR,FUNNEL_AM,FUNNEL_REFRNC_NO,BRAND_NR,SPLIT_PARMTR_TX,CONTNT_GROUP_TX,CONTNT_SUBGRP_TX,PAGE_TITLE_TX,PAGE_NM,PAGE_ID,USER_JOURNY_TX,EVENT_TYPE_CD,APP_SCREEN_NM,APP_SCREEN_CD,APP_PBLSHR_NR,APP_CATGRY_NR,APP_NM,APP_VERSIN_NO,APP_STATUS_NR,APP_EVENT_TS,USER_AGENT_TX,DEVICE_TYPE_NR,DEVICE_NM,DEVICE_TX,APP_DEVICE_NM,APP_DEVICE_TX,APP_OPRTNG_SYSTEM_TX,SCREEN_SIZE_TX,BROWSR_SIZE_TX,LANGUG_CD,TIME_ZONE_NO,CONTRY_CD,COMPNY_NM,MOBILE_NETWRK_NR,QUESTN_TX,ANSWER_TX,ANSWER_TYPE_CD,CUSTOM_PARMTR_LIST_TX,QUERY_TX,SEARCH_ENGINE_NM,PAID_SEARCH_IN,SEARCH_TX,TRAFFC_SOURCE_TX,TRAFFC_TYPE_TX,site_nr,site_brand_nr,SESSION_START,SESSION_END,SURE_IN,DEVICE,rand
2016-01-18 09:12:15,2016-01-18,09:12:15,edb42672-3e10-4a3e-bf79-aafd209539a1,WEBTRENDS_ID=edb42672-3e10-4a3e-bf79-aafd209539a1; WT_ACCT=edb42672-3e10-4a3e-bf79-aafd209539a1,,9225dd30,dcsfn00jp100000w4d2tx3zos_2b3p,2b3p,1,secure.lloydsbank.co.uk,/personal/a/logon/entermemorableinformation.jsp,https://secure.lloydsbank.co.uk/personal/a/logon/entermemorableinformation.jsp,/personal/a/logon/entermemorableinformation.jsp,secure.lloydsbank.co.uk/personal/a/logon/entermemorableinformation.jsp,https://online.lloydsbank.co.uk/personal/logon/login.jsp?WT.ac=PLO0512,10.4.11,,,,,,,,,,,,Lloyds,personal,a,logon,Enter Memorable Information,,,,0,,,,,,,,,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko,Computer,,,,,,1093x614,1093x490,en-GB,0,united kingdom,,,,,,brand,WT.z_testdom=0,,,,,,SECURE,LLoyds,,,,,0.72351326664
2016-01-21 17:44:13,2016-01-21,17:44:13,132a83b3-da05-4dc0-b27b-84ca569ad027,-,,c0e3d485,dcsvm5u5510000wk5f6ky6x4w_3w4j,3w4j,,,errorloggedout,app/errorloggedout,errorloggedout,,-,,,,,,,,,,,,,,lloydsmobilebankingnga,login,welcome to online banking,error logged out,,,,0,errorloggedout,s007,lloydsbankinggroup,lloydsnextgenapp,nextgenapp,ip8.0-8.0.552,live,1453398251.352861,"Mozilla/5.0 (iPhone; CPU iPhone OS 9_2 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Mobile/13C75",Mobile Phone,iPhone,iPhone,ipad3 wifi,"ipad3,1",9.2,960x640,,en_gb,0,united kingdom,,,,,,,,,,,,,NGA NATIVE,LLoyds,,,,,0.374509199087
2016-01-20 21:50:35,2016-01-20,21:50:35,2220d1474049fb482e41453213574279,WEBTRENDS_ID=28e38a81-33ff-40b4-9397-aba10ddba3fc; WT_ACCT=28e38a81-33ff-40b4-9397-aba10ddba3fc,,d36ea765,dcsxby8tz00000w8oh3utaci2_8p2n,8p2n,1,online.lloydsbank.co.uk,/personal/logon/login.jsp,https://online.lloydsbank.co.uk/personal/logon/login.jsp,/personal/logon/login.jsp,online.lloydsbank.co.uk/personal/logon/login.jsp,https://secure.lloydsbank.co.uk/personal/a/mobile/account_details_ress/OWEGXWFPRK2YXTJ2VOTFQ7HYVRZLXPQKOKUMLJRMQ5COIGSUP4WQ/WCCLTC6CDXY6UTQDQSBIAVWTD3VNBKWFCYB3YVA/62LCDBKEY6GJW/,8.6.2,,,,,mobileLogin,1,,,,,,,IB;mobilebanking,Mobile Banking,loginwithreglink,Lloyds Bank - Mobile Banking - Login,,,,0,,,,,,,,,"Mozilla/5.0 (Linux; Android 5.1.1; D5503 Build/14.6.A.1.236) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36",Mobile Phone,Xperia Z1 Compact,Xperia Z1 Compact,,,,360x640,360x511,en-GB,0,united kingdom,,,,,,,target=/personal/a/accountenquiry/mobile/viewaccountoverview/viewproductdetailsmobileress.jsp?lnkcmd=ifCommercial%3AifCustomerBar%3AifMobLO%3AoutputLinkLogOutMobile&al=&SmartAppBannerSwitch=Y&SmartAppBanner=Y,,,,,,MOBILE SECURE,LLoyds,,,,,0.651408162142
2016-01-26 20:37:13,2016-01-26,20:37:13,d01c6fb3-49c0-4522-9714-0b0fcd8e55e1,-,,b45cc65a,dcsvm5u5510000wk5f6ky6x4w_3w4j,3w4j,,,loginsplashscreen,app/loginsplashscreen,loginsplashscreen,,-,,,,,,otp,0,,,,,,,lloydsmobilebankingnga,login,otp,login splash screen,,,,0,loginsplashscreen,l001,lloydsbankinggroup,lloydsnextgenapp,nextgenapp,ip8.0-8.0.552,live,1453840626.655496,"Mozilla/5.

Ok now lets put into dataframes and merge the two

In [21]:
df1 = bq.Query(frauds).to_dataframe()
len(df1)

44250

In [25]:
df2 = bq.Query(nonfrauds).to_dataframe()
len(df2)

50000

In [22]:
df3 = df1.append(df2)

Lets create the binary fraud identifier

In [31]:
df3['Fraud'] = np.where(df3['SESSION_START']=='Jan 1, 1970, 12:00:00 AM', '1', '0')

Now lets define the X and Y and check the bias

In [40]:
Y = df3['Fraud']
X = df3.drop('Fraud', axis=1)

In [47]:
yFreq = scipy.stats.itemfreq(Y)
print(yFreq)

[['0' 94250]]
